In [1]:
import pprint
import pandas
import sqlite3
import re
import tqdm

processed_cedict = []
with open('cedict_1_0_ts_utf-8_mdbg.txt', 'r', encoding='utf-8') as f:
    for line in f:
        if line.startswith('#'):
            continue
        line = line.strip()
        if not line:
            continue
        # space, space, bracket, slash
        pinyin = line.split('[', 1)[1].split(']', 1)[0]
        assert all(
            c in '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZüÜ:,· ' for c in pinyin)
        english = line.split('/', 1)[1]
        trad, simp = line.split(' ', 2)[:2]
        pinyin = pinyin.replace('u:', 'ü')
        pinyin = pinyin.replace('U:', 'Ü')
        english = english.strip('/')
        # english = english.replace('/', ' * ')
        processed_cedict.append((trad, simp, pinyin, english))
df = pandas.DataFrame(processed_cedict, columns=[
                      'Traditional', 'Simplified', 'Pinyin', 'English'])
df['Definitions'] = df['English'].apply(lambda x: len(x.split('/')))


chinese_unicode_ranges = {
    'CJK Unified Ideographs': [0x4E00, 0x9FFF],
    'CJK Unified Ideographs Extension A': [0x3400, 0x4DBF],
    'CJK Unified Ideographs Extension B': [0x20000, 0x2A6DF],
    'CJK Unified Ideographs Extension C': [0x2A700, 0x2B73F],
    'CJK Unified Ideographs Extension D': [0x2B740, 0x2B81F],
    'CJK Unified Ideographs Extension E': [0x2B820, 0x2CEAF],
    'CJK Unified Ideographs Extension F': [0x2CEB0, 0x2EBEF],
    'CJK Compatibility Ideographs': [0xF900, 0xFAFF],
    'CJK Compatibility Ideographs Supplement': [0x2F800, 0x2FA1F],
}

def is_chinese_char(char):
    char = ord(char)
    for _, (start, end) in chinese_unicode_ranges.items():
        if start <= char <= end:
            return True
    return False


In [4]:
df.sort_values('Definitions', ascending=False, inplace=True)
print(df)

       Traditional Simplified               Pinyin  \
74462            白          白                 bai2   
110022           門          门                 men2   
95271            行          行                xing2   
120874           點          点                dian3   
46289            掉          掉                diao4   
...            ...        ...                  ...   
103723          辦貨         办货            ban4 huo4   
117778         驗光師        验光师     yan4 guang1 shi1   
91299           茫崖         茫崖            Mang2 ya2   
103722          辦罪         办罪            ban4 zui4   
121605         𰻞𰻞麵        𰻝𰻝面  biang2 biang2 mian4   

                                                  English  Definitions  
74462   white/snowy/pure/bright/empty/blank/plain/clea...           21  
110022  gate/door/CL:扇[shan4]/gateway/doorway/CL:個|个[g...           21  
95271   to walk/to go/to travel/a visit/temporary/make...           19  
120874  point/dot/drop/speck/o'clock/point (in space o...  

In [8]:

def is_chinese(word):
    return all(is_chinese_char(char) for char in word)


def is_english(word):
    return all(u'\u0041' <= char <= u'\u005a' or u'\u0061' <= char <= u'\u007a' for char in word)


def has_punctuation(word):
    return any(char in '.,?!:;-/' for char in word)


def any_chinese(word):
    return any(u'\u4e00' <= char <= u'\u9fff' for char in word)


non_chinese_trad = df[~df['Traditional'].apply(is_chinese)]
non_chinese_simp = df[~df['Simplified'].apply(is_chinese)]
non_english = df[~df['English'].apply(is_english)]
pinyin_chinese = df[df['Pinyin'].apply(any_chinese)]

only_english_punctuation = df[~df['Traditional'].apply(has_punctuation) & ~df['Simplified'].apply(
    has_punctuation) & ~df['Pinyin'].apply(has_punctuation)]


In [ ]:
print('Total:', len(df))
print('Non-Chinese Traditional:', len(non_chinese_trad))
print('Non-Chinese Simplified:', len(non_chinese_simp))
print('Non-English:', len(non_english))
print('Pinyin Chinese:', len(pinyin_chinese))
print('Only English Punctuation:', len(only_english_punctuation))


In [ ]:
print(non_chinese_trad)
non_chinese_trad.to_csv('non_chinese_trad.csv', index=False)


In [52]:
conn = sqlite3.connect('cedict.db')
c = conn.cursor()


def equivalent_chinese(traditional, simplified):
    for i in range(len(traditional)):
        traditional_char = traditional[i]
        simplified_char = simplified[i]
        if traditional_char != simplified_char:
            traditional_lookup = c.execute(
                'SELECT * FROM cedict_lookup WHERE Lookup = ?', traditional_char).fetchall()
            simplified_lookup = c.execute(
                'SELECT * FROM cedict_lookup WHERE Lookup = ?', simplified_char).fetchall()
            traditional_cedict_ids = set([row[2] for row in traditional_lookup])
            simplified_cedict_ids = set([row[2] for row in simplified_lookup])
            if not traditional_cedict_ids.intersection(simplified_cedict_ids):
                return False
    return True


def pipe_radius(text):
    pipe_circles = []
    pipe_indices = [i for i, char in enumerate(text) if char == '|']
    if len(pipe_indices) == 0:
        return pipe_circles
    max_pipe_radii = []
    for pipe_index in pipe_indices:
        for i in range(1, len(text)):
            if pipe_index - i < 0 or pipe_index + i >= len(text):
                max_pipe_radii += [i - 1]
                break
            if text[pipe_index - i] == '|' or text[pipe_index + i] == '|':
                max_pipe_radii += [i - 2]
                break
    pipe_index_with_max_radii = zip(pipe_indices, max_pipe_radii)
    for pipe_index, max_pipe_radius in pipe_index_with_max_radii:
        for i in range(max_pipe_radius, 1, -1):
            traditional_index = pipe_index - i
            simplified_index = pipe_index + i
            traditional_text = text[traditional_index:pipe_index]
            simplified_text = text[pipe_index + 1:simplified_index + 1]
            print(traditional_text, simplified_text)
            if equivalent_chinese(traditional_text, simplified_text):
                pipe_circles += [(traditional_text, simplified_text)]
                break
    return pipe_circles


variant_df['pipe_circles'] = variant_df['Variant'].apply(pipe_radius)
variant_df.to_csv('variant_pipe_circles.csv', index=False)


三個臭皮匠，賽過一個諸葛亮 三个臭皮匠，赛过一个诸葛亮
解鈴還須繫鈴人 解铃还须系铃人
身份證號碼 身份证号码
太平洋週邊 太平洋周边
弗吉尼亞州 弗吉尼亚州
安達曼群島 安达曼群岛
小兒麻痺 小儿麻痹
返璞歸真 返璞归真
戰略伙伴 战略伙伴
唯利是圖 唯利是图
敢做敢當 敢做敢当
開國元勳 开国元勋
氣宇軒昂 气宇轩昂
發人深省 发人深省
絡腮鬍子 络腮胡子
百廢俱興 百废俱兴
畢恭畢敬 毕恭毕敬
繃扒吊拷 绷扒吊拷
直截了當 直截了当
信口開河 信口开河
蘇門答臘 苏门答腊
勾心鬥角 勾心斗角
一錘定音 一锤定音
沒有勁頭 没有劲头
借資挹注 借资挹注
噼裡啪啦 噼里啪啦
驕奢淫逸 骄奢淫逸
意識形態 意识形态
蒙在鼓裡 蒙在鼓里
石嘴山區 石嘴山区
憤憤不平 愤愤不平
一股腦兒 一股脑儿
沉魚落雁 沉鱼落雁
一筆抹殺 一笔抹杀
小題大做 小题大做
亞美尼亞 亚美尼亚
全神貫注 全神贯注
阿德萊德 阿德莱德
殺一儆百 杀一儆百
興高采烈 兴高采烈
死乞白賴 死乞白赖
五筆字型 五笔字型
五勞七傷 五劳七伤
邀買人心 邀买人心
歸根結底 归根结底
烏七八糟 乌七八糟
再接再厲 再接再厉
腦性麻痺 脑性麻痹
德國戰車 德国战车
金櫃石室 金柜石室
念念有詞 念念有词
正經八百 正经八百
噼裡啪啦 噼里啪啦
探頭探腦 探头探脑
頭昏腦脹 头昏脑胀
頭暈腦脹 头晕脑胀
軼事遺聞 轶事遗闻
弦誦不輟 弦诵不辍
轟動一時 轰动一时
欲取姑與 欲取姑与
開誠布公 开诚布公
想當然爾 想当然尔
長年累月 长年累月
新婚燕爾 新婚燕尔
蒙在鼓裡 蒙在鼓里
貿易伙伴 贸易伙伴
炸土豆條 炸土豆条
盤根錯節 盘根错节
無動於衷 无动于衷
神鵰俠侶 神雕侠侣
齊大非偶 齐大非偶
殺氣騰騰 杀气腾腾
鋌而走險 铤而走险
平白無故 平白无故
無依無靠 无依无靠
糊裡糊塗 糊里糊涂
貪得無厭 贪得无厌
波濤粼粼 波涛粼粼
索馬利亞 索马利亚
嘮嘮叨叨 唠唠叨叨
橫浜 横浜
鄧小平 邓小平
頂呱呱 顶呱呱
沒人味 没人味
一口氣 一口气
奧黛麗 奥黛丽
未知數 未知数
混球兒 混球儿
頭半天 头半天
邱吉爾 邱吉尔
出婁子 出娄子
節骨眼 节骨眼
邁阿密 迈阿密
出婁子 出娄子
鹹津津 咸津津
撥浪鼓 拨浪鼓
鹹絲絲 咸丝丝
不對勁 不对劲
不大離 不大离
莊稼戶 庄稼户
奧巴馬 奥巴马
紀錄

C:\Users\bnm\AppData\Local\Temp\ipykernel_31100\2809921454.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variant_df['pipe_circles'] = variant_df['Variant'].apply(pipe_radius)


In [40]:
df.to_json('cedict.json', orient='records', force_ascii=False)


In [5]:
# create sqlite database
conn = sqlite3.connect('cedict.db')
c = conn.cursor()


In [6]:
# drop all tables
c.execute('DROP TABLE IF EXISTS cedict;')
c.execute('DROP TABLE IF EXISTS cedict_lookup;')
conn.commit()

In [7]:
c.execute('''CREATE TABLE cedict (
    ID INTEGER PRIMARY KEY AUTOINCREMENT, 
    Traditional TEXT, 
    Simplified TEXT, 
    Pinyin TEXT, 
    English TEXT,
    Definitions INTEGER)''')
conn.commit()
df.to_sql('cedict', conn, if_exists='append', index=False)


In [8]:
c.execute('''CREATE TABLE cedict_lookup (
    id INTEGER PRIMARY KEY AUTOINCREMENT, 
    Lookup TEXT, 
    cedict_id INTEGER,
    UNIQUE(Lookup, cedict_id),
    FOREIGN KEY(cedict_id) REFERENCES cedict(ID))''')
c.execute('CREATE INDEX lookup_index ON cedict_lookup(Lookup)')
conn.commit()


In [9]:
cedict_query = 'INSERT INTO cedict_lookup (Lookup, cedict_id) VALUES (?, ?)'
cedict_query += ' ON CONFLICT (Lookup, cedict_id) DO NOTHING'
print(cedict_query)
for row in c.execute('SELECT ID, Traditional, Simplified FROM cedict').fetchall():
    c.execute(cedict_query, (row[1], row[0]))
    c.execute(cedict_query, (row[2], row[0]))
conn.commit()


INSERT INTO cedict_lookup (Lookup, cedict_id) VALUES (?, ?) ON CONFLICT (Lookup, cedict_id) DO NOTHING


In [10]:
def get_variant_sentences(text: str):
    descriptions = text.split(',')
    descriptions = re.split(r',|\|/|]', text)
    text.split()
    for description in descriptions:
        description = description.lower().strip()
        if 'variant of' in description:
            print(description)
            return description.split('variant of')[1].strip()

conn = sqlite3.connect('cedict.db')
c = conn.cursor()

def lookup_sqlite(word):
    c.execute("SELECT * FROM cedict_lookup WHERE lookup = ?", (word,))
    results = c.fetchall()
    result_cedict_ids = [result[2] for result in results]
    return result_cedict_ids

def insert_into_cedict_lookup(row):
    variant_lookup_result = row['variant_lookup_result']
    traditional = row['Traditional']
    simplified = row['Simplified']
    for result in variant_lookup_result:
        c.execute(cedict_query, (traditional, result))
        c.execute(cedict_query, (simplified, result))

# variant in english
variant_df = df[df['English'].str.contains('variant of')]
print(variant_df)
variant_df['variant_sentence'] = variant_df['English'].apply(get_variant_sentences)
def extract_chinese(word):
    return ''.join(char for char in word if is_chinese_char(char) or char in '，|')
variant_df['Variant'] = variant_df['variant_sentence'].apply(extract_chinese)
variant_df['variant_length'] = variant_df['Variant'].apply(len)
variant_df.sort_values(by=['variant_length'], inplace=True, ascending=False)
variant_df.drop(columns=['variant_length'], inplace=True)
variant_df['variant_characters'] = variant_df['Variant'].apply(lambda x: len([x for x in x if x == '|']))
variant_df.sort_values(by=['variant_characters'], inplace=True, ascending=False)
# only lookup with traditional
variant_df['variant_lookup'] = variant_df['Variant'].apply(lambda x: x.split('|')[0])
variant_df['variant_lookup_result'] = variant_df['variant_lookup'].apply(lookup_sqlite)
variant_df['lookup_result_count'] = variant_df['variant_lookup_result'].apply(len)
variant_df.sort_values(by=['lookup_result_count'], inplace=True, ascending=False)
variant_df.to_csv('variant.csv', index=False)


cedict_query = 'INSERT INTO cedict_lookup (Lookup, cedict_id) VALUES (?, ?) ON CONFLICT DO NOTHING'

variant_df.apply(insert_into_cedict_lookup, axis=1)
conn.commit()

       Traditional Simplified     Pinyin  \
47523            搭          搭        da1   
65521            溫          温       wen1   
19267            合          合        he2   
96430            襄          襄     xiang1   
7235             佚          佚        yi4   
...            ...        ...        ...   
103677           辠          罪       zui4   
103678           辢          辣        la4   
91253           茅竹         茅竹  mao2 zhu2   
91269            茊          茊        zi1   
91278            茘          荔        li4   

                                                  English  Definitions  
47523   to put up/to build (scaffolding)/to hang (clot...           11  
65521   warm/lukewarm/temperature/to warm up/mild/soft...           10  
19267   to close/to join/to fit/to be equal to/whole/t...           10  
96430   to help/to assist/mutual assistance/to rush in...           10  
7235    lost/missing/forsaken/dissolute/(of a woman) b...            9  
...                              

C:\Users\asdfuiop\AppData\Local\Temp\ipykernel_5300\2620481696.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variant_df['variant_sentence'] = variant_df['English'].apply(get_variant_sentences)
C:\Users\asdfuiop\AppData\Local\Temp\ipykernel_5300\2620481696.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variant_df['Variant'] = variant_df['variant_sentence'].apply(extract_chinese)
C:\Users\asdfuiop\AppData\Local\Temp\ipykernel_5300\2620481696.py:35: SettingWithCopyWarning: 
A value is trying to be

In [ ]:
def convert_windows_path_to_unix(path):
    return path.replace('\\', '/')


path = r"""

"""
path = convert_windows_path_to_unix(path)
print(path)


In [17]:
import sqlite3
def check_db(path):
    conn = sqlite3.connect(path)
    c = conn.cursor()
    c.execute('SELECT * FROM cedict WHERE Simplified = "的"')
    print(c.fetchall())
    c.execute('SELECT * FROM cedict_lookup WHERE Lookup = "的"')
    print(c.fetchall())
    c.execute('SELECT * FROM cedict_lookup WHERE Lookup = "龢"')
    print(c.fetchall())
    # correct variant can be selected by choosing the look up with the lowest id because of the way that the db is created
    sql_query = 'SELECT * FROM cedict_lookup JOIN cedict ON cedict_lookup.cedict_id = cedict.ID WHERE cedict_lookup.Lookup = "龢" ORDER BY cedict_lookup.id ASC LIMIT 1'
    c.execute(sql_query)
    print(c.fetchall())


check_db('cedict.db')


[(1725, '的', '的', 'de5', "of; ~'s (possessive particle)/(used after an attribute)/(used to form a nominal expression)/(used at the end of a declarative sentence for emphasis)/also pr. [di4] or [di5] in poetry and songs", 5), (77115, '的', '的', 'di2', 'really and truly', 1), (77121, '的', '的', 'di1', 'see 的士[di1 shi4]', 1), (77122, '的', '的', 'di4', "(bound form) bull's-eye; target", 1)]
[(3449, '的', 1725), (154229, '的', 77115), (154241, '的', 77121), (154243, '的', 77122)]
[(243245, '龢', 1042), (243247, '龢', 4007), (243249, '龢', 14183), (243251, '龢', 30234), (83023, '龢', 41512), (83729, '龢', 41865), (243255, '龢', 103506), (243257, '龢', 103604), (243259, '龢', 103679)]
[(83023, '龢', 41512, 41512, '龢', '和', 'he2', 'old variant of 和[he2]/harmonious', 2)]
